# Network Science With NetworkX
## Penguicon 2019
Contains code derived from [_Network Science in Python with NetworkX Quick Start Guide_](https://www.packtpub.com/big-data-and-business-intelligence/network-science-python-and-networkx-quick-start-guide) distrubuted under the MIT License. See `LICENSE` for more information.

<p><img style="float:left; padding-right: 2em;" src="https://d255esdrn735hr.cloudfront.net/sites/default/files/imagecache/ppv4_main_book_cover/cover_36.png"/>
<span style="font-size: 2em;">bit.ly/NetworkXBook</span></p>

## Setup
### Download code

    git clone https://github.com/elplatt/Penguicon-2019-Networks.git
    cd Penguicon-2019-Networks
    jupyter lab

#### Import libraries

In [ ]:
# Configure plotting in Jupyter
from matplotlib import pyplot as plt
%matplotlib inline

# Import NetworkX
import networkx as nx

#### (Optional) Configure plotting parameters and random number generation

In [ ]:
plt.rcParams.update({
    'figure.figsize': (16, 9),
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False})
# Seed random number generator
import random
from numpy import random as nprand
seed = hash("Network Science in Python") % 2**32
nprand.seed(seed)
random.seed(seed)

## Your first network

In [ ]:
nx.draw_networkx(nx.karate_club_graph())

## Building Networks

In [ ]:
# Create an empty network
G = nx.Graph()
G

In [ ]:
# Add some nodes
G.add_node('Detroit')
G.add_node('Chicago')
G.add_node('Milwaukee')

In [ ]:
nx.draw_networkx(G, node_color='#afafff', node_size=4000)

In [ ]:
# Add some edges
G.add_edge('Detroit', 'Chicago')
G.add_edge('Chicago', 'Milwaukee')

In [ ]:
nx.draw_networkx(G, node_color='#afafff', node_size=4000)

In [ ]:
G.add_nodes_from([
    'Toledo',
    'Albany',
    'Boston',
    'New York'
])

In [ ]:
G.add_edges_from([
    ('Detroit', 'Toledo'),
    ('Chicago', 'Toledo'),
    ('Toledo', 'Albany'),
    ('Albany', 'Boston'),
    ('Albany', 'New York'),
    ('Boston', 'New York')
])

In [ ]:
nx.draw_networkx(G, node_color='#afafff', node_size=4000)

## Exploring networks

In [ ]:
# List nodes
G.nodes

In [ ]:
# List edges
G.edges

In [ ]:
# Iterating through edges
for v, w in G.edges:
    print('{} is connected to {}'.format(v, w))

In [ ]:
# Finding neighbors
G.neighbors('Detroit')

In [ ]:
list(G.neighbors('Detroit'))

## Annotating nodes and edges

In [ ]:
# Add node with additional data
G.add_node('Detroit', address='11 W Baltimore Ave, Detroit, MI')

In [ ]:
# Annotate existing node with data
G.nodes['Chicago']['address'] = '225 S Canal St, Chicago, IL'

In [ ]:
# Annotate existing edge with data
G.edges['Detroit', 'Chicago']['route'] = 'Wolverine Limited'

In [ ]:
G.node['Detroit']

In [ ]:
G.edges['Detroit', 'Chicago']

## Reading networks from files

In [ ]:
# Download file
import urllib
url = 'https://www.macalester.edu/~abeverid/data/stormofswords.csv'
response = urllib.request.urlopen(url)
data = response.read()
with open('stormofswords.csv', 'wb') as f:
    f.write(data)

In [ ]:
G = nx.Graph()
with open('stormofswords.csv') as f:
    f.readline()
    rows = list(f)
    for row in rows:
        v, w, weight = row.split(",")
        G.add_edge(v, w, weight=int(weight))

In [ ]:
plt.figure(figsize=(16, 16))
nx.draw_networkx(
    G,
    weight=weight,
    node_color='#afafff',
    node_size=1000,
    edge_color='#999999',
    font_size=8,)

## Centrality

In [ ]:
# Get betweenness for all nodes
betweenness = nx.betweenness_centrality(G)
# Find 10 highest betweenness nodes
sorted(betweenness.items(), key=lambda x: x[1], reverse=True)[:10]

In [ ]:
# Get page_rank for all nodes
page_rank = nx.pagerank(G)
# Find 10 highest page_rank nodes
sorted(page_rank.items(), key=lambda x: x[1], reverse=True)[:10]

## Community detection

In [ ]:
# Import the community module
import networkx.algorithms.community as nxcom
# Find the communities
G = nx.karate_club_graph()
communities = nxcom.greedy_modularity_communities(G)
# Show the communities
for com in communities:
    print(sorted(com))

In [ ]:
def get_color(i, r_off=1, g_off=1, b_off=1):
    '''Return the ith element in a sequence of distinguishible colors.'''
    r0, g0, b0 = 0, 0, 0
    n = 16
    low, high = 0.1, 0.9
    span = high - low
    r = low + span * (((i + r_off) * 3) % n) / (n - 1)
    g = low + span * (((i + g_off) * 5) % n) / (n - 1)
    b = low + span * (((i + b_off) * 7) % n) / (n - 1)
    return (r, g, b)

In [ ]:
def set_node_community(G, communities):
    '''Add community to node attributes'''
    for c, v_c in enumerate(communities):
        for v in v_c:
            # Add 1 to save 0 for external edges
            G.nodes[v]['community'] = c + 1
            
def set_edge_community(G):
    '''Find internal edges and add their community to their attributes'''
    for v, w, in G.edges:
        if G.nodes[v]['community'] == G.nodes[w]['community']:
            # Internal edge, mark with community
            G.edges[v, w]['community'] = G.nodes[v]['community']
        else:
            # External edge, mark as 0
            G.edges[v, w]['community'] = 0

In [ ]:
# Set node and edge communities
set_node_community(G, communities)
set_edge_community(G)

# Set community color for nodes
node_color = [
    get_color(G.nodes[v]['community'])
    for v in G.nodes]

# Set community color for internal edges
external = [
    (v, w)
    for v, w in G.edges
    if G.edges[v, w]['community'] == 0]
internal = [(v, w)
            for v, w in G.edges
            if G.edges[v, w]['community'] > 0]
internal_color = [
    get_color(G.edges[e]['community'])
    for e in internal]

In [ ]:
karate_pos = nx.spring_layout(G)
# Draw external edges
nx.draw_networkx(
    G,
    pos=karate_pos,
    node_size=0,
    edgelist=external,
    edge_color="#666666")
# Draw nodes and internal edges
nx.draw_networkx(
    G,
    pos=karate_pos,
    node_color=node_color,
    edgelist=internal,
    edge_color=internal_color)

## Your own social network

### Load data

In [ ]:
import json

def load_lost_circles_json(in_file):
    '''Create a Graph from LostCircles json'''
    with open(in_file) as f:
        raw = json.load(f)
    id_to_name = dict(
        (i, datum["name"])
        for i, datum in enumerate(raw['nodes']))
    edges = [
        (id_to_name[datum["source"]], id_to_name[datum["target"]])
        for datum in raw['links']]
    G = nx.Graph()
    G.add_edges_from(edges)
    return G

G = load_lost_circles_json('egonet.json')

## Connected component

In [ ]:
G = nx.subgraph(G, max(nx.connected_components(G), key=len))

### Find communities

In [ ]:
communities = nxcom.greedy_modularity_communities(G)

In [ ]:
# Set node and edge communities
set_node_community(G, communities)
set_edge_community(G)

# Set community color for nodes
node_color = [
    get_color(G.nodes[v]['community'])
    for v in G.nodes]

# Set community color for internal edges
external = [
    (v, w)
    for v, w in G.edges
    if G.edges[v, w]['community'] == 0]
internal = [(v, w)
            for v, w in G.edges
            if G.edges[v, w]['community'] > 0]
internal_color = [
    get_color(G.edges[e]['community'])
    for e in internal]

In [ ]:
pos = nx.spring_layout(G)
# Draw external edges
nx.draw_networkx(
    G,
    pos=pos,
    node_size=0,
    edgelist=external,
    edge_color="#666666",
    with_labels=False)
# Draw internal edges
nx.draw_networkx(
    G,
    pos=pos,
    node_size=0,
    edgelist=internal,
    edge_color=internal_color,
    alpha=0.3,
    with_labels=False)

## Thanks!
<p><img style="float:left; padding-right: 2em;" src="https://d255esdrn735hr.cloudfront.net/sites/default/files/imagecache/ppv4_main_book_cover/cover_36.png"/>
    <span style="font-size: 2em;">elplatt.com<br/>@elplatt@greatjustice.net<br/>bit.ly/NetworkXBook</span></p>